In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from joblib import dump, load

module_path = os.path.abspath(os.path.join('..', 'code'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import utils

## Procesado de datos Oferta de cursos

In [3]:
df = utils.load_data('../data/courses.csv')

In [4]:
def initial_filter(df):
    df = df[df['Lugar']=='CAMPUS PRINCIPAL']
    # Mantener solo los edificos usados para la investigación
    df = df[df['Edificio'].str.contains(r'.*SD.*|.*\.centro.*|.*ML.*|.*LL.*', na=False)]
    # Eliminar filas duplicadas
    df.drop_duplicates(inplace=True)
    
    return df

In [5]:
def further_processing(df):
    # Mantener solo las columnas necesarias
    df = df[['Curso','NRC','Periodo','Edificio','Salón','Días','Horas', 'Inscritos']]
    # Eliminar cursos sin inscritos
    df = df[df['Inscritos']>0]
    return df

In [6]:
def save_file(processed_df, path):
    processed_df.to_csv(path, index=False)

In [7]:
def process_file(input_path, output_path):
    df = utils.load_data(input_path)
    df = initial_filter(df)
    df = further_processing(df)
    save_file(df, output_path)

In [8]:
dump(process_file, '../models/process_file.joblib')

['../models/process_file.joblib']

In [9]:
process_file_loaded = load('../models/process_file.joblib')

process_file_loaded('../data/courses.csv', '../data/processed_courses.csv')



### Usando los datos de talanqueras

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
courses = pd.read_csv('../data/processed_courses.csv')
talanqueras = pd.read_csv('../data/output.csv')

In [3]:
courses.shape

(4457, 8)

In [4]:
talanqueras['Edificio'] = talanqueras['Puerta'].str.split('-', expand=True)[0]
talanqueras['Edificio'] = talanqueras['Edificio'].str.replace(r'[0-9]', '', regex=True)

courses['Salón'] = courses['Salón'].str.replace('.','')
courses['Edificio'] = courses['Salón'].str.split('_', expand=True)[0]

In [5]:
courses.sample(10)

,Curso,NRC,Periodo,Edificio,Salón,Días,Horas,Inscritos
2010,IIND-1001 PROGRAMA DE ACOMPAÑAMIENTO,66051,202420,ML,ML_119,I,1100-1220,8
2089,IIND-2109T TRABAJO ASISTIDO SISTEMAS APOYO A L...,40023,202420,ML,ML_510,V,0800-0920,17
2353,IQYA-3050 SEMINARIO DE PROYECTO,63018,202420,RGD,RGD_004,I,1230-1350,21
1128,ECON-3321C COMPLEMENTARIA ECONOMETRÍA 1,73183,202420,ML,ML_107,V,0800-0920,31
1457,ESCR-1102 ESCRITURA UNIVERSITARIA II (CICLO 2 ...,66921,202420,SD,SD_303,J,1800-1950,20
1097,ECON-2221C COMPLEMENTARIA INTRODUCCIÓN A LA MA...,73170,202420,LL,LL_302,V,0930-1050,24
1701,IBIO-2340L LABORATORIO DE FUNDAMENTOS DEL MACH...,69346,202420,RGD,RGD_203,I,1400-1520,30
4250,MMER-4345 SIMULACIÓN DE MERCADEO (CICLO 2 DE 8...,71592,202420,SD,SD_804,J,1800-2050,29
4039,MGAP-4418 ECOLOGIA INDUSTRIAL,71536,202420,SD,SD_702,J,1100-1350,24
391,CBCA-1329 FILOSOFÍA DE LOS SUEÑOS Y ADIVINACIÓ...,71929,202420,RGD,RGD_306-7,L,1600-1750,45


In [6]:
def count_entradas(edificio):
    return talanqueras['Edificio'].str.contains(edificio).sum()

In [7]:
edif = pd.DataFrame()
edif['Edificio'] = courses['Edificio'].unique()
for edificio in edif['Edificio']:
    edif.loc[edif['Edificio']==edificio, 'Entradas'] = count_entradas(edificio)
edif.head()  #  Conteo de entradas por edificio

,Edificio,Entradas
0,RGD,10088.0
1,ML,35062.0
2,SD,35198.0
3,LL,10014.0


In [11]:
# df para contar entradas por rangos de horas
hours_count = pd.DataFrame()
hours_count['Edificio'] = courses['Edificio']
hours_count['Rango de Horas'] = talanqueras['FechaHora'].str.split(' ', expand=True)[1].str.split(':', expand=True)[0].astype(float) 

In [12]:
hours_count.head()

,Edificio,Rango de Horas
0,RGD,7.0
1,RGD,6.0
2,ML,7.0
3,ML,12.0
4,RGD,15.0


In [15]:
import numpy as np

def create_ranges(df):
    earliest = float(hours_count['Rango de Horas'].min())
    latest = float(hours_count['Rango de Horas'].max())
    df['Rango de Horas'] = pd.cut(df['Rango de Horas'], bins=np.arange(earliest, latest, 0.5), right=False)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

create_ranges(hours_count)

,Edificio,Rango de Horas
0,RGD,"[7.0, 7.5)"
1,RGD,"[6.0, 6.5)"
2,ML,"[7.0, 7.5)"
3,ML,"[12.0, 12.5)"
4,RGD,"[15.0, 15.5)"
...,...,...
86,ML,"[4.0, 4.5)"
87,LL,"[2.0, 2.5)"
88,RGD,"[2.0, 2.5)"
89,LL,"[1.0, 1.5)"


In [16]:
#count entries by hour
hours_count['Entradas'] = 0
for i in range(len(hours_count)):
    edificio = hours_count['Edificio'].iloc[i]
    left = hours_count['Rango de Horas'].iloc[i].left
    right = hours_count['Rango de Horas'].iloc[i].right
    for j in range(len(talanqueras)):
        hora = int(talanqueras['FechaHora'].iloc[j].split(' ')[1].split(':')[0])
        building = talanqueras['Edificio'].iloc[j]
        if (hora == left or hora == right) and building == edificio:
            hours_count['Entradas'].iloc[i] += 1

C:\Users\carlo\AppData\Local\Temp\ipykernel_704\2067999010.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hours_count['Entradas'].iloc[i] += 1


In [17]:
#Ordenar por edificio y rango de horas
hours_count = hours_count.sort_values(by=["Edificio", "Rango de Horas"])

In [18]:
hours_count

,Edificio,Rango de Horas,Entradas
90,LL,"[0.0, 0.5)",22
89,LL,"[1.0, 1.5)",34
87,LL,"[2.0, 2.5)",27
68,LL,"[3.0, 3.5)",26
83,LL,"[4.0, 4.5)",25
...,...,...,...
35,SD,"[18.0, 18.5)",1043
50,SD,"[19.0, 19.5)",1087
30,SD,"[20.0, 20.5)",957
16,SD,"[21.0, 21.5)",681


In [19]:
# Exportamos los 2 df a csv
edif.to_csv('../data/entradas_edificio.csv', index=False)
hours_count.to_csv('../data/entradas_edificio_hora.csv', index=False)

## Agrupacion de datos usando oferta de cursos

In [20]:
new = courses.copy()

In [21]:
new

,Curso,NRC,Periodo,Edificio,Salón,Días,Horas,Inscritos
0,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,25454,202420,RGD,RGD_206-7,L,0630-0750,43
1,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,25454,202420,RGD,RGD_206-7,I,0630-0750,43
2,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,11411,202420,ML,ML_515,I,0800-0920,46
3,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,11411,202420,ML,ML_606,L,0800-0920,46
4,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,11411,202420,RGD,RGD_112-13,I,0800-0920,46
...,...,...,...,...,...,...,...,...
4452,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,RGD,RGD_311,V,1400-1650,11
4453,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,RGD,RGD_311,V,1400-1650,11
4454,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,RGD,RGD_311,V,1400-1650,11
4455,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,RGD,RGD_311,V,1400-1650,11


In [22]:
new.drop(columns=['Curso','NRC','Periodo','Salón', 'Días'], inplace=True)

In [23]:
new.sort_values(by=['Edificio','Horas'], inplace=True)
new.drop_duplicates(subset=['Edificio','Horas'], keep='first', inplace=True)
new.reset_index(drop=True, inplace=True)

In [24]:
def convert_to_range(horas):
    # Split the start and end times
    start, end = horas.split("-")

    start_hour = int(start[:2])  # First two digits (hours)
    end_hour = int(end[:2])      # First two digits (hours)
    if end[2:] != '00':
        end_hour += 1

    return[start_hour, end_hour]

# Apply the conversion function
new["Rango de Horas"] = new["Horas"].apply(convert_to_range)

In [25]:
new.drop(columns=['Horas','Rango de Horas'], inplace=True)

In [26]:
#Nos da el numero de estudiantes inscritos en un curso por edificio
new = new.groupby('Edificio').count()*new['Inscritos'].sum()

In [27]:
new

,Inscritos
Edificio,
LL,256168
ML,374880
RGD,331144
SD,374880


In [28]:
new.to_csv('../data/inscritos_edificio.csv')